In [1]:
import pandas as pd
import numpy as np
import pickle 
#from sklearn.decomposition import NMF
#from sklearn.impute import KNNImputer

## Load in the data

In [2]:
movies = pd.read_csv('./ml-latest-small/movies.csv', index_col=0)
links = pd.read_csv('./ml-latest-small/links.csv', index_col=0) ### don't know if we need this one
ratings = pd.read_csv('./ml-latest-small/ratings.csv',  index_col=1, parse_dates=True)
tags = pd.read_csv('./ml-latest-small/tags.csv', index_col=1, parse_dates=True)

In [3]:
movies.columns, links.columns, ratings.columns, tags.columns

(Index(['title', 'genres'], dtype='object'),
 Index(['imdbId', 'tmdbId'], dtype='object'),
 Index(['userId', 'rating', 'timestamp'], dtype='object'),
 Index(['userId', 'tag', 'timestamp'], dtype='object'))

In [4]:
movies.shape, links.shape, ratings.shape, tags.shape

((9742, 2), (9742, 2), (100836, 3), (3683, 3))

In [5]:
links.head()

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0
4,114885,31357.0
5,113041,11862.0


In [6]:
ratings.head()

,userId,rating,timestamp
movieId,,,
1,1,4.0,964982703
3,1,4.0,964981247
6,1,4.0,964982224
47,1,5.0,964983815
50,1,5.0,964982931


In [7]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [8]:
tags.head()

,userId,tag,timestamp
movieId,,,
60756,2,funny,1445714994
60756,2,Highly quotable,1445714996
60756,2,will ferrell,1445714992
89774,2,Boxing story,1445715207
89774,2,MMA,1445715200


In [9]:
movies.dtypes

title     object
genres    object
dtype: object

In [10]:
tags.dtypes

userId        int64
tag          object
timestamp     int64
dtype: object

In [11]:
 movies['genres'].str.split(pat='|', n=- 1, expand=True)

,0,1,2,3,4,5,6,7,8,9
movieId,,,,,,,,,,
1,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
2,Adventure,Children,Fantasy,None,None,None,None,None,None,None
3,Comedy,Romance,None,None,None,None,None,None,None,None
4,Comedy,Drama,Romance,None,None,None,None,None,None,None
5,Comedy,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
193581,Action,Animation,Comedy,Fantasy,None,None,None,None,None,None
193583,Animation,Comedy,Fantasy,None,None,None,None,None,None,None
193585,Drama,None,None,None,None,None,None,None,None,None


In [18]:
genres = pd.concat((movies['title'],movies['genres'].str.split(pat='|', n=- 1, expand=True)),
                    axis=1)
#.str.get_dummies(sep='|')),axis=1)
genres

,title,0,1,2,3,4,5,6,7,8,9
movieId,,,,,,,,,,,
1,Toy Story (1995),Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
2,Jumanji (1995),Adventure,Children,Fantasy,None,None,None,None,None,None,None
3,Grumpier Old Men (1995),Comedy,Romance,None,None,None,None,None,None,None,None
4,Waiting to Exhale (1995),Comedy,Drama,Romance,None,None,None,None,None,None,None
5,Father of the Bride Part II (1995),Comedy,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action,Animation,Comedy,Fantasy,None,None,None,None,None,None
193583,No Game No Life: Zero (2017),Animation,Comedy,Fantasy,None,None,None,None,None,None,None
193585,Flint (2017),Drama,None,None,None,None,None,None,None,None,None


In [19]:
with open('genres.pickle', 'wb') as f:
    pickle.dump(genres, f)

In [13]:
genres.columns

Index(['title', '(no genres listed)', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'],
      dtype='object')

In [14]:
genres['num_genres'] = genres.iloc[:,2:].sum(axis=1)
genres

,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,num_genres
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,2
4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,3
5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4
193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
193585,Flint (2017),0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [15]:
genres['(no genres listed)'].sum()

34

In [16]:
genres[genres['num_genres']==0].shape

(34, 22)

---
---
## Prep for Movie PCS

- idea: combine all DFs that contain "info" re movies into one DF
- perform PCA to result in one dimension(value?) per movie

#### To Dos:
1. try to fix the datetime
2. pivot/melt the "tags" table


In [17]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings

,userId,rating,timestamp
movieId,,,
1,1,4.0,2000-07-30 18:45:03
3,1,4.0,2000-07-30 18:20:47
6,1,4.0,2000-07-30 18:37:04
47,1,5.0,2000-07-30 19:03:35
50,1,5.0,2000-07-30 18:48:51
...,...,...,...
166534,610,4.0,2017-05-03 21:53:22
168248,610,5.0,2017-05-03 22:21:31
168250,610,5.0,2017-05-08 19:50:47


In [18]:
tags['timestamp'] = pd.to_datetime(tags['timestamp'], unit='s')
tags

,userId,tag,timestamp
movieId,,,
60756,2,funny,2015-10-24 19:29:54
60756,2,Highly quotable,2015-10-24 19:29:56
60756,2,will ferrell,2015-10-24 19:29:52
89774,2,Boxing story,2015-10-24 19:33:27
89774,2,MMA,2015-10-24 19:33:20
...,...,...,...
7382,606,for katie,2007-02-11 22:46:59
7936,606,austere,2007-03-08 22:18:54
3265,610,gun fu,2017-05-03 20:39:44


In [19]:
movies.columns, links.columns, ratings.columns, tags.columns, genres.columns

(Index(['title', 'genres'], dtype='object'),
 Index(['imdbId', 'tmdbId'], dtype='object'),
 Index(['userId', 'rating', 'timestamp'], dtype='object'),
 Index(['userId', 'tag', 'timestamp'], dtype='object'),
 Index(['title', '(no genres listed)', 'Action', 'Adventure', 'Animation',
        'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
        'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
        'Sci-Fi', 'Thriller', 'War', 'Western', 'num_genres'],
       dtype='object'))

----
## 2.

In [20]:
tags.head()

,userId,tag,timestamp
movieId,,,
60756,2,funny,2015-10-24 19:29:54
60756,2,Highly quotable,2015-10-24 19:29:56
60756,2,will ferrell,2015-10-24 19:29:52
89774,2,Boxing story,2015-10-24 19:33:27
89774,2,MMA,2015-10-24 19:33:20


In [30]:
tags_dum = tags.groupby('movieId')['tag'].value_counts().unstack()
tags_dum

tag,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
tags_dummy = tags['tag'].str.get_dummies()
tags_dummy

,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
60756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7382,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
#merged = 
pd.merge(genres, tags_dummy, how='outer', on='movieId')

,title,(no genres listed),Action_x,Adventure_x,Animation_x,Children,Comedy_x,Crime,Documentary_x,Drama,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Toy Story (1995),0,0,1,1,1,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Jumanji (1995),0,0,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,Flint (2017),0,0,0,0,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
